In [2]:
from sys import platform
if platform == "linux":
    # linux/
    MAIN_DIR='C:/Users/soufiane.chami/Desktop/PhysioNet/PhysioNet2019/'
    
elif platform == "darwin":
        # OS X
    MAIN_DIR='/Users/macbook/Desktop/Submission_PhysioNet/'
    
elif platform == "win32":
        # Windows...
    MAIN_DIR='C:/Users/soufiane.chami/Desktop/PhysioNet/PhysioNet2019/'
    
    
import os
import pandas as pd
import numpy as np
import sys
from sys import platform
from IPython.display import display, HTML
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import time
from sklearn.preprocessing import normalize
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
import time 
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)


import seaborn as sns
from matplotlib import pyplot as plt
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["Feature", "importance"]].groupby("Feature").mean().sort_values(by="importance", ascending=False)[:10].index
    best_features = feature_importance_df_[["Feature", "importance"]].groupby("Feature").mean().sort_values(by="importance", ascending=False)[:35]
    best_features.reset_index(inplace=True)
    print(best_features.dtypes)
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="Feature", data=best_features)
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

    
def load_data(list_of_files, INP_DIR, return_featues=False):
    data = []
    for file_name in tqdm(list_of_files): 
        subject= pd.read_csv(INP_DIR+ file_name, sep='|')
        subject['time_flow']= subject.index.values + 1
        subject['Subject_ID'] = file_name.split('.')[0]
        subject = subject.set_index(['Subject_ID', 'time_flow'])
        subject["SepsisLabel"]= subject.SepsisLabel.values.max()
        y_cols =["SepsisLabel"]
        features = [f for f in subject.columns if f not in y_cols+ ['ICULOS']]
        subject= subject.interpolate()
        subject.fillna(method='ffill', inplace=True)
        subject.fillna(method='bfill', inplace=True)
        subject.fillna(0, inplace=True)
        data.append(subject.reset_index())
        
    data = pd.concat(data, axis=0, ignore_index=True)
#     data.set_index(['Subject_ID','time_flow'], inplace=True)
    target = data[['Subject_ID','SepsisLabel']]
#     del data['SepsisLabel']
    if return_featues: 
        return data,target, features
    else: 
        return data,target
    
TRAIN_Info= MAIN_DIR +'setA'
TEST_Info = MAIN_DIR +'setB/'
TRAIN_DIR= MAIN_DIR +'training_setA/'
TEST_DIR = MAIN_DIR +'training_setB/'
train_files = os.listdir(TRAIN_Info)
test_files = os.listdir(TEST_DIR)

# TRAIN 
# outputs will have the following shape
# df_train: (790215, 39), target:(790215, 2)
# 9.6517% of sepsis subjects
training_setA= pd.read_csv(MAIN_DIR +'setA'+'/'+'info_training_'+'setA'+'.csv')
yes_sepsis_setA= pd.read_csv(MAIN_DIR +'setA'+'/'+'yes_epsis_subject_id'+'setA'+'.csv')
no_sepsis_setA= pd.read_csv(MAIN_DIR +'setA'+'/'+'no_Sepsis_subject_id'+'setA'+'.csv')
print(str(yes_sepsis_setA.shape[0]/no_sepsis_setA.shape[0]*100))

# TEST 
# outputs will have the following shape
# df_train: (761995, 39), target: (761995, 2)
# 6.0558% of sepsis subjects
training_setB= pd.read_csv(MAIN_DIR +'setB'+'/'+'info_training_'+'setB'+'.csv')
yes_sepsis_setB= pd.read_csv(MAIN_DIR +'setB'+'/'+'yes_epsis_subject_id'+'setB'+'.csv')
no_sepsis_setB= pd.read_csv(MAIN_DIR +'setB'+'/'+'no_Sepsis_subject_id'+'setB'+'.csv')
print(str(yes_sepsis_setB.shape[0]/no_sepsis_setB.shape[0]*100))

start_from_scratch=False

if start_from_scratch:
    
    df_train , trn_target, features = load_data(training_setA.subject_id.values+'.psv',TRAIN_DIR,return_featues=True)
    
    df_train.to_csv("df_train_untouched.csv", index=False)
    trn_target.to_csv("target_train_untouched.csv", index=False)
    
    df_test , target_test = load_data(training_setB.subject_id.values+'.psv',TEST_DIR,return_featues=False)
    
    df_test.to_csv("df_test_untouched.csv", index=False)
    target_test.to_csv("target_test_untouched.csv", index=False)

else: 
    MAIN_DIR= '/Users/macbook/OneDrive - North Dakota University System/PhysioNet Scripts/2019/Notebooks/LGBM/'
    df_train= pd.read_csv("df_train_untouched.csv")
    trn_target= pd.read_csv("target_train_untouched.csv")
    
    df_test= pd.read_csv("df_test_untouched.csv")
    target_test= pd.read_csv("target_test_untouched.csv")

    features= [f for f in df_train.columns if f not in ['ICULOS', 'SepsisLabel', 'Subject_ID']]

    
    
is_sepsis = training_setA.SepsisLabel.values
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

param = {'num_leaves': 120,
        'metric': 'auc',
         'objective': 'binary',
         'is_unbalance': True,  
         'seed': 820,
         'max_depth': -1,
         'n_estimators': 90,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         'eval_metric': 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 4590}

for fold_, (trn_idx, val_idx) in enumerate(folds.split(training_setA.subject_id.values,is_sepsis)):
    print("fold {}".format(fold_))
    
    
    # load data based on the fold's indexes 
    
    ##### train
    trn_idx = df_train['Subject_ID'].isin(training_setA.subject_id.values[trn_idx].tolist())
    
    trn_data= lgb.Dataset(df_train[features][trn_idx], label=trn_target['SepsisLabel'][trn_idx].values)
    
    
    ##### validation
    val_idx = df_train['Subject_ID'].isin(training_setA.subject_id.values[val_idx].tolist())
    
    val_data= lgb.Dataset(df_train[features][val_idx], label=trn_target['SepsisLabel'][val_idx].values)
    
  
    # train lgb
    num_round = 1000
    clf = lgb.train(param, trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100, early_stopping_rounds = 100)
        
    oof[val_idx] = clf.predict(df_train[features][val_idx],
                               num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    


display_importances(feature_importance_df)    





9.651676911463388
6.055785343090466
fold 0
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[90]	training's auc: 0.95714	valid_1's auc: 0.876135
fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.962014	valid_1's auc: 0.870588
[200]	training's auc: 0.975406	valid_1's auc: 0.874786
[300]	training's auc: 0.984217	valid_1's auc: 0.876858
[400]	training's auc: 0.989614	valid_1's auc: 0.878141
[500]	training's auc: 0.993299	valid_1's auc: 0.878855
[600]	training's auc: 0.995726	valid_1's auc: 0.878863
Early stopping, best iteration is:
[547]	training's auc: 0.994603	valid_1's auc: 0.879023
fold 2
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.958101	valid_1's auc: 0.874688
[200]	training's auc: 0.972511	valid_1's auc: 0.879059
[300]	training's auc: 0.981704	valid_1's auc: 0.882853
[400]	training's auc: 0.987491	valid_1's auc: 0.885904
[500]	trainin

In [24]:
clf.save_model('lgb_classifier.txt', num_iteration=clf.best_iteration)
model = lgb.Booster(model_file='lgb_classifier.txt')

## Generate predictions

In [ ]:
#!/usr/bin/env python

import sys
import numpy as np, pandas as pd
import os, shutil, zipfile


def build_data(engine, time, x, max_time, is_test):


    return out_x, out_y

    



def load_sepsis_model():
    
#     from keras.models import load_model
    model = lgb.Booster(model_file='lgb_classifier.txt')
    
    return model


def load_challenge_data(file):
    subject=  pd.read_csv(file, sep = "|")
    subject= pd.read_csv(INP_DIR+ file_name, sep='|')
    subject['time_flow']= subject.index.values + 1
    subject['Subject_ID'] = file_name.split('.')[0]
    subject = subject.set_index(['Subject_ID', 'time_flow'])
    subject["SepsisLabel"]= subject.SepsisLabel.values.max()
    y_cols =["SepsisLabel"]
    features = [f for f in subject.columns if f not in y_cols+ ['ICULOS']]
    subject= subject.interpolate()
    subject.fillna(method='ffill', inplace=True)
    subject.fillna(method='bfill', inplace=True)
    subject.fillna(0, inplace=True)

    return subject





def get_sepsis_score(test, model):

#     print("data dhas been built .......")
    # Make some predictions and put them alongside the real TTE and event indicator values
    test_predict = 1- model.predict(test)
    labels = (test_predict>0.4).astype(int)
    
    return scores, labels
 


import numpy as np, os, sys
# from get_sepsis_score import load_sepsis_model, get_sepsis_score

def save_challenge_predictions(file, scores, labels):
    with open(file, 'w') as f:
        f.write('PredictedProbability|PredictedLabel\n')
        for (s, l) in zip(scores, labels):
            f.write('%g|%d\n' % (s, l))

if __name__ == '__main__':
    # Parse arguments.
    if len(sys.argv) != 3:
        raise Exception('Include the input and output directories as arguments, e.g., python driver.py input output.')

    input_directory = '/Users/macbook/Desktop/Submission_PhysioNet/training_setB/'  #sys.argv[1]
    output_directory = '/Users/macbook/Desktop/Submission_PhysioNet/predictions_setB/' #sys.argv[2]

    # Find files.
    files = []
    for f in os.listdir(input_directory):
        if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('psv'):
            files.append(f)

    if not os.path.isdir(output_directory):
        os.mkdir(output_directory)

    # Load model.
    model = load_sepsis_model()
    files = sorted(files)
    # Iterate over files.
    
    from tqdm import tqdm
    for f in tqdm(files):
        # Load data.
        input_file = os.path.join(input_directory, f)
        data = load_challenge_data(input_file)

        # Make predictions.
        num_rows = len(data)
        scores = np.zeros(num_rows)
        labels = np.zeros(num_rows)
        current_data = data
        current_score, current_label = get_sepsis_score(current_data, model)

        scores= current_score
        labels= current_label
        # Save results.
        output_file = os.path.join(output_directory, f)
        save_challenge_predictions(output_file, scores, labels)
        

In [43]:
def load_challenge_data(file):
    subject=  pd.read_csv(file, sep = "|")
    subject['time_flow']= subject.index.values + 1
    subject['Subject_ID'] = file.split('/')[-1].split('.')[0]
    subject = subject.set_index(['Subject_ID', 'time_flow'])
    subject["SepsisLabel"]= subject.SepsisLabel.values.max()
    y_cols =["SepsisLabel"]
    features = [f for f in subject.columns if f not in y_cols+ ['ICULOS']]
    subject= subject.interpolate()
    subject.fillna(method='ffill', inplace=True)
    subject.fillna(method='bfill', inplace=True)
    subject.fillna(0, inplace=True)

    return subject


test_df= load_challenge_data('/Users/macbook/Desktop/Submission_PhysioNet/training_setB/p100001.psv')


array([0.15025147, 0.15025147, 0.15139939, 0.15139939, 0.14854631,
       0.15139939, 0.15139939, 0.14854631, 0.14854631, 0.14854631,
       0.15139939, 0.14854631, 0.14854631, 0.14854631, 0.15139939,
       0.14854631, 0.14854631, 0.15139939, 0.15139939, 0.14854631,
       0.14854631, 0.15139939, 0.15139939, 0.15025147])